# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 12 - Processamento de Linguagem Natural</font>

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Obs: Este é um material de bônus incluído neste curso. PyTorch é estudado em detalhes no curso <a href="https://www.datascienceacademy.com.br/course?courseid=deep-learning-frameworks">Deep Learning Frameworks</a> e aplicado em PLN no curso <a href="https://www.datascienceacademy.com.br/course?courseid=processamento-de-linguagem-natural-e-reconhecimento-de-voz">Processamento de Linguagem Natural</a>.

### Estudo de Caso - Buscador de Palavras em Texto Por Similaridade

![title](imagens/buscador.png)

**A definição deste estudo de caso está no manual em pdf no Capítulo 12 do Curso de <a href="https://www.datascienceacademy.com.br/course?courseid=machine-learning-engineer">Machine Learning</a>**. 

Faça a leitura do manual antes de prosseguir com o Estudo de Caso.

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/home/daholive/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/home/daholive/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/daholive/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/home/daholive/anaconda3/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above except

In [ ]:
# Instala o PyTorch
!pip install -q torch 

In [2]:
# Imports
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.autograd import Variable
from nltk.tokenize import word_tokenize
%matplotlib inline
torch.manual_seed(1)

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

torch      1.6.0
numpy      1.19.1
matplotlib 3.3.1
Data Science Academy


### Carregando e Processando os Dados

Para este estudo de caso, usaremos o famoso texto de Isaac Asimov: The Last Question.

http://users.ece.cmu.edu/~gamvrosi/thelastq.html

Traduzimos o texto e usaremos para treinar o modelo GloVe e depois buscar palavras por similaridade. Recomendados a leitura do arquivo asimov.txt (usado na célula abaixo) antes de executar o restante do Jupyter Notebook.

In [4]:
# Abre o arquivo para leitura e carrega na variável arquivo_texto
arquivo_texto = open('dados/asimov.txt', 'r')

In [5]:
# Converte as palavars para minúsculo
texto = arquivo_texto.read().lower()

In [6]:
# Fecha o arquivo
arquivo_texto.close()

In [7]:
# Tokenização do texto
texto_token = word_tokenize(texto)

In [8]:
# Variável para o comprimento total dos tokens
comp_tokens = len(texto_token)

In [9]:
print("Número de Tokens: ", comp_tokens)

Número de Tokens:  5282


### Criando o Vocabulário

In [10]:
# Criando o vocabulário
vocab = set(texto_token)
vocab_size = len(vocab)
print("Tamanho do Vocabulário:", vocab_size)

Tamanho do Vocabulário: 1397


In [16]:
# Dicionário para mapear as palavras aos índices
palavra_indice = {palavra: i for i, palavra in enumerate(vocab)}
#palavra_indice

In [17]:
# Dicionário para mapear os índices às palavras
indice_palavra = {i: palavra for i, palavra in enumerate(vocab)}
#indice_palavra

Salvo indicação em contrário, usamos um contexto de dez palavras à esquerda e dez palavras à direita.

In [14]:
# Tamanho do contexto
CONTEXT_SIZE = 10

In [15]:
# Matriz de co-ocorrência preenchida com zeros
co_occ_mat = np.zeros((vocab_size, vocab_size))
co_occ_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Agora percorremos os dicionários de mapeamento criados anteriormente e preenchemos a matriz de co-ocorrência.

In [18]:
# Loop externo por todo comprimento do vocabulário
for i in range(comp_tokens):
    
    # Loop interno pelo tamanho do contexto
    for dist in range(1, CONTEXT_SIZE + 1):
        
        # Obtém o índice do token
        ix = palavra_indice[texto_token[i]]
        
        # Se a palara estiver à esquerda, inserimos à esquerda na matriz de co-ocorrência
        if i - dist > 0:
            left_ix = palavra_indice[texto_token[i - dist]]
            co_occ_mat[ix, left_ix] += 1.0 / dist
            
        # Se a palara estiver à direita, inserimos à direita na matriz de co-ocorrência
        if i + dist < len(texto_token):
            right_ix = palavra_indice[texto_token[i + dist]]
            co_occ_mat[ix, right_ix] += 1.0 / dist

In [19]:
# Matriz de co-ocorrência
co_occ_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
# Transposta da matriz de co-ocorrências
# Retorna um array 2-D com uma linha para cada elemento não-zero 
co_occs = np.transpose(np.nonzero(co_occ_mat))

In [21]:
# Print
print("Shape da Matriz de Co-Ocorrência:", co_occ_mat.shape)

Shape da Matriz de Co-Ocorrência: (1397, 1397)


In [22]:
# Print
print("Matriz de Co-Ocorrência Não-Zero:\n", co_occs)

Matriz de Co-Ocorrência Não-Zero:
 [[   0  384]
 [   0  393]
 [   0  587]
 ...
 [1396 1271]
 [1396 1342]
 [1396 1366]]


### Criando o Modelo

In [23]:
# Tamanho da embedding
EMBEDDING_SIZE = 50

In [24]:
# Hiperparâmetros
X_MAX = 100
ALPHA = 0.75
BATCH_SIZE = 32
LEARNING_RATE = 0.05
EPOCHS = 200

In [25]:
# Classe para o modelo
class Glove(nn.Module):

    # Método construtor
    def __init__(self, vocab_size, comat, embedding_size, x_max, alpha):
        super(Glove, self).__init__()
        
        # Matriz de embeddings com as palavras centrais
        self.embedding_V = nn.Embedding(vocab_size, embedding_size)
        
        # Matriz de embeddings com as palavras de contexto
        self.embedding_U = nn.Embedding(vocab_size, embedding_size)

        # Bias
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
        # Inicializa os parâmtetros (pesos que a rede aprende durante o treinamento)
        for params in self.parameters():
            nn.init.uniform_(params, a = -0.5, b = 0.5)
            
        # Define os hiperparâmetros (que controlam o treinamento)
        self.x_max = x_max
        self.alpha = alpha
        self.comat = comat
    
    # Função de forward
    def forward(self, center_word_lookup, context_word_lookup):
        
        # Matrizes embedding de pesos para centro e contexto
        center_embed = self.embedding_V(center_word_lookup)
        target_embed = self.embedding_U(context_word_lookup)

        # Matrizes embedding de bias para centro e contexto
        center_bias = self.v_bias(center_word_lookup).squeeze(1)
        target_bias = self.u_bias(context_word_lookup).squeeze(1)

        # Elementos da matriz de co-ocorrência
        co_occurrences = torch.tensor([self.comat[center_word_lookup[i].item(), context_word_lookup[i].item()]
                                       for i in range(BATCH_SIZE)])
        
        # Carrega os pesos
        weights = torch.tensor([self.weight_fn(var) for var in co_occurrences])

        # Funçã de perda
        loss = torch.sum(torch.pow((torch.sum(center_embed * target_embed, dim = 1)
            + center_bias + target_bias) - torch.log(co_occurrences), 2) * weights)
        
        return loss
       
    # Definição do peso
    def weight_fn(self, x):
        if x < self.x_max:
            return (x / self.x_max) ** self.alpha
        return 1
        
    # Soma de V e U como nossos vetores de palavras
    def embeddings(self):
        return self.embedding_V.weight.data + self.embedding_U.weight.data

In [26]:
# Função para gerar um bacth de palavras
def gera_batch(model, batch_size = BATCH_SIZE):
    
    # Extrai uma amostra
    sample = np.random.choice(np.arange(len(co_occs)), size = batch_size, replace = False)
    
    # Listas de vetores
    v_vecs_ix, u_vecs_ix = [], []
    
    # Loop pela amostra para gerar os vetores
    for chosen in sample:
        ind = tuple(co_occs[chosen])  
        
        lookup_ix_v = ind[0]
        lookup_ix_u = ind[1]
        
        v_vecs_ix.append(lookup_ix_v)
        u_vecs_ix.append(lookup_ix_u) 
        
    return torch.tensor(v_vecs_ix), torch.tensor(u_vecs_ix)

### Treinamento do Modelo

In [27]:
# Função para o treinamento
def treina_glove(comat):
    
    # Lista para os erros
    losses = []
    
    # Cria o modelo Glove
    model = Glove(vocab_size, comat, embedding_size = EMBEDDING_SIZE, x_max = X_MAX, alpha = ALPHA)
    
    # Otimizador
    optimizer = optim.Adagrad(model.parameters(), lr = LEARNING_RATE)
    
    # Loop pelo número de épocas
    for epoch in range(EPOCHS):
        
        # Erro total
        total_loss = 0
        
        # Número de bacthes
        num_batches = int(len(texto_token) / BATCH_SIZE)
        
        # Loop pelos batches
        for batch in tqdm(range(num_batches)):
            
            # Zera os gradientes do modelo
            model.zero_grad()
            
            # Obtém o bacth de dados
            data = gera_batch(model, BATCH_SIZE)
            
            # Calcula o erro
            loss = model(*data)
            
            # Executa o backpropagation
            loss.backward()
            
            # Otimiza os pesos (aqui é onde ocorre o aprendizado)
            optimizer.step()
            
            # Erro total para a epoch
            total_loss += loss.item()
            
        # Erros do modelo
        losses.append(total_loss)
        
        # Print da epoch e erro médio do modelo
        print('Epoch : %d, Erro Médio : %.02f' % (epoch, np.mean(losses)))
        
    return model, losses 

In [ ]:
# Executa a função de treinamento e retorna o modelo e os erros
model, losses = treina_glove(co_occ_mat)

  9%|▉         | 15/165 [00:00<00:01, 144.30it/s]

Epoch : 0, Erro Médio : 207.09


 12%|█▏        | 20/165 [00:00<00:00, 192.92it/s]

Epoch : 1, Erro Médio : 189.49


 12%|█▏        | 19/165 [00:00<00:00, 189.86it/s]

...

 11%|█         | 18/165 [00:00<00:00, 173.85it/s]

Epoch : 43, Erro Médio : 81.94


 11%|█         | 18/165 [00:00<00:00, 172.68it/s]

Epoch : 44, Erro Médio : 81.02


 87%|████████▋ | 144/165 [00:00<00:00, 159.80it/s]

In [ ]:
# Função para o plot do erro durante o treinamento
def plot_loss(losses, title):
    plt.plot(range(len(losses)), losses)
    plt.xlabel('Epoch')
    plt.ylabel('Erro')
    plt.title(title)
    plt.figure()

In [ ]:
# Plot
plot_loss(losses, "Erro de Treinamento do Modelo GloVe")

### Testando o Modelo: Similaridade de Palavras, analogias de palavras

In [ ]:
# Função que retorna a embedding de uma palavra
def get_palavra(palavra, modelo, word_to_ix):
    return model.embeddings()[word_to_ix[palavra]]

In [ ]:
# Função para busca a palavra mais próxima
def busca_palavra_similaridade(vec, word_to_ix, n = 10):
    all_dists = [(w, torch.dist(vec, get_palavra(w, model, palavra_indice))) for w in palavra_indice]
    return sorted(all_dists, key = lambda t: t[1])[:n]

In [ ]:
# Gerando o vetor (embedding) de uma palavra 
vector = get_palavra("espaço", model, palavra_indice)
print(vector)

In [ ]:
# Busca as palavras similares à palavra "espaço"
busca_palavra_similaridade(vector, palavra_indice)

Observe que a palavra "espaço" tem 0 de distância para si mesma. A próxima palavra mais parecida com "espaço" é "universo" e assim por diante. Quanto menor a distância, mais parecida a palavra. Lembrando que a busca por similaridade é feita com as embeddings treinadas com o modelo GloVe. 

Mais um exemplo:

In [ ]:
# Gerando o vetor (embedding) de uma palavra 
vector = get_palavra("solar", model, palavra_indice)
print(vector)

In [ ]:
# Busca as palavras similares à palavra "solar"
busca_palavra_similaridade(vector, palavra_indice)

A distância da palavra "solar" para si mesma é 0 e a palavra com maior similaridade é "energia" o que faz todo sentido se você leu o texto do Asimov usado para treinar o modelo.

### Analogia

![title](imagens/glove.png)

Observe na imagem acima que criamos uma "fórmula" com 3 palavras visando buscar a quarta palavra, o que é feito por analogia das embeddings (vetores de palavras).

Criamos então uma função para buscar a palavra por analogia no formato: 

palavra1 : palavra2 :: palavra3 : ?

In [ ]:
# Função para busca de palavra por analogia
def busca_analogia(p1, p2, p3, n = 5, filtro = True):
    
    # Print
    print('\n[%s : %s :: %s : ?]' % (p1, p2, p3))
   
    # p2 - p1 + p3 = p4
    closest_words = busca_palavra_similaridade(get_palavra(p2, model, palavra_indice) - 
                                               get_palavra(p1, model, palavra_indice) + 
                                               get_palavra(p3, model, palavra_indice), 
                                               palavra_indice)
    
    # Vamos excluir as 3 palavras passadas como parâmetro
    if filtro:
        closest_words = [t for t in closest_words if t[0] not in [p1, p2, p3]]
        
    for tuple in closest_words[:n]:
        print('(%.4f) %s' % (tuple[1], tuple[0]))

In [ ]:
# Busca por analogia
busca_analogia("família", "crianças", "humano")

E aí estão as palavras que melhor se encaixam na quarta palavra, de acordo com nosso modelo.

Quanto maior a distância, menor a similaridade! Treine o modelo com seus próprios textos e experimente a busca por similaridade.

# Fim